Hipotesis: "Los ataques de tiburón mortales disminuyen mientras la película de "Tiburón" está en cartelera"

In [446]:
import pandas as pd

import re as regex

from functions import *



In [447]:
df = pd.read_csv("Inputs/Dataset_Shark_Attacks.csv", encoding = "ISO-8859-1")

# pd.set_option('display.max_rows', 6000)
# pd.set_option('display.max_columns', 500)

# display(df)


In [448]:
#Observamos la envergadura y principales detalles del Dataset

#display(f"La dimensión del dataset son {df.shape[0]} filas y {df.shape[1]} columnas, mientras que sus columnas son: {list(df.columns)}")

# df.head(5)

# df.tail(5)

# df.shape

# df.columns

In [449]:
renaming = {df.columns[9]: 'Género'}

df2 = df.rename(columns=renaming)

df3 = df2.rename(columns={'Case Number':'Caso','Date':'Fecha','Year':'Años','Type':'Ataque', 'Country':'País', 'Area':'Área', 'Location':'Localización','Activity':'Actividad', 'Age':'Edad', 'Injury':'Herida', 'Fatal (Y/N)':'Mortalidad'})


In [450]:
df4 = df3[['Caso','Fecha','Años','Ataque','País','Área','Localización','Actividad','Género','Edad','Herida','Mortalidad']]

In [451]:

df4.Caso = df4["Caso"].astype(str).apply(case_filtering).dropna()


/usr/lib/python3/dist-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [452]:
df4 = df4[["Años","Caso","Género", "Mortalidad"]]

df4.sort_values("Años",ascending=False).dropna().set_index("Años")

,Caso,Género,Mortalidad
Años,,,
2016,18-09-2016,M,N
2016,05-06-2016,F,Y
2016,31-03-2016,F,N
2016,07-04-2016,M,N
2016,07-04-2016,M,N
2016,08-04-2016,M,N
2016,09-04-2016,F,Y
2016,13-04-2016,M,N
2016,18-04-2016,M,N


In [453]:
df4.Supervivencia = df4["Mortalidad"].astype(str)

df4.Supervivencia.dtypes



/home/alex/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.


dtype('O')

In [454]:


df4.Mortalidad = df4["Mortalidad"].astype(str).apply(m_correction)

df4.loc[df4['Mortalidad'] == "0"]




,Años,Caso,Género,Mortalidad
0,2016,18-09-2016,M,0
1,2016,18-09-2016,M,0
2,2016,18-09-2016,M,0
3,2016,17-09-2016,M,0
4,2016,15-09-2016,M,0
5,2016,15-09-2016,NaN,0
6,2016,11-09-2016,M,0
7,2016,07-09-2016,F,0
9,2016,05-09-2016,F,0
10,2016,05-09-2016,M,0


In [455]:
df4 = df4.iloc[0:5325].dropna()

df4

,Años,Caso,Género,Mortalidad
0,2016,18-09-2016,M,0
1,2016,18-09-2016,M,0
2,2016,18-09-2016,M,0
3,2016,17-09-2016,M,0
4,2016,15-09-2016,M,0
6,2016,11-09-2016,M,0
7,2016,07-09-2016,F,0
8,2016,06-09-2016,M,1
9,2016,05-09-2016,F,0
10,2016,05-09-2016,M,0


In [456]:
df4["Mortalidad"] = df4["Mortalidad"].astype("int64")

df4["Género"] = df4["Género"].astype(str)


In [457]:
df4['Hombres'] = df4.Género.apply(lambda x: x=="M")

df4['Mujeres'] = df4.Género.apply(lambda x: x=="F")

df4 = df4[["Años","Caso","Hombres","Mujeres","Mortalidad"]]

def genero_fixing(x):

    if x == True:
        return 1

    else:
        return 0


df4['Mujeres'] = df4.Mujeres.apply(genero_fixing)

df4['Hombres'] = df4.Hombres.apply(genero_fixing)

df4

/home/alex/.local/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/home/alex/.local/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Años,Caso,Hombres,Mujeres,Mortalidad
0,2016,18-09-2016,1,0,0
1,2016,18-09-2016,1,0,0
2,2016,18-09-2016,1,0,0
3,2016,17-09-2016,1,0,0
4,2016,15-09-2016,1,0,0
6,2016,11-09-2016,1,0,0
7,2016,07-09-2016,0,1,0
8,2016,06-09-2016,1,0,1
9,2016,05-09-2016,0,1,0
10,2016,05-09-2016,1,0,0


In [458]:
df4 = df4.groupby(["Años"]).agg({'Caso':'count', 'Hombres': 'sum', 'Mujeres': 'sum', 'Mortalidad':'sum'}).dropna().reset_index()

df4

,Años,Caso,Hombres,Mujeres,Mortalidad
0,1848,1,1,0,1
1,1900,11,11,0,5
2,1901,7,7,0,3
3,1902,15,15,0,7
4,1903,10,10,0,8
5,1904,13,13,0,9
6,1905,14,14,0,9
7,1906,21,21,0,14
8,1907,23,20,3,13
9,1908,11,9,1,10


In [459]:
df5 = df4

df5.reset_index()

df5


,Años,Caso,Hombres,Mujeres,Mortalidad
0,1848,1,1,0,1
1,1900,11,11,0,5
2,1901,7,7,0,3
3,1902,15,15,0,7
4,1903,10,10,0,8
5,1904,13,13,0,9
6,1905,14,14,0,9
7,1906,21,21,0,14
8,1907,23,20,3,13
9,1908,11,9,1,10


In [460]:
fd = pd.read_csv("Inputs/Dataset_Movies.csv", encoding = "ISO-8859-1")


/home/alex/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [461]:
fd.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [462]:
fd2 = fd.rename(columns = {"release_date":"Lanzamiento","original_title":"Título"})

fd3 = fd2[["Lanzamiento","Título"]]

In [463]:
fd3.Lanzamiento = fd3["Lanzamiento"].astype(str).apply(case_filtering)


In [464]:
# fd4[fd4['Título'].astype(str).str.contains("shark")]

fd3.Título = fd3["Título"].astype(str).apply(find_word)

fd4 = fd3.dropna(axis=0, subset=['Lanzamiento', 'Título'])

fd4["Años"] =  fd4["Lanzamiento"].astype(str).apply(extract_year)

fd4["Años"] = fd4["Años"].astype(int)

fd4



/home/alex/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/home/alex/.local/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,Lanzamiento,Título,Años
318,10-09-1994,swimming with sharks,1994
1338,18-06-1975,jaws,1975
1339,16-06-1978,jaws 2,1978
1340,16-03-1983,jaws 3-d,1983
3998,17-07-1987,jaws: the revenge,1987
5716,18-12-1981,sharky's machine,1981
8180,20-09-2004,shark tale,2004
10105,10-06-2005,the adventures of sharkboy and lavagirl,2005
11787,11-09-2006,sharkwater,2006
11839,15-07-2007,eagle vs shark,2007


In [465]:
fd4 = fd4[["Años", "Lanzamiento", "Título"]]

fd4.sort_values("Años", ascending=False).dropna().reset_index()

# .set_index("Años")

,index,Años,Lanzamiento,Título
0,39771,2016,31-07-2016,sharknado 4: the 4th awakens
1,35515,2015,20-07-2015,3-headed shark attack
2,43722,2015,23-03-2015,shark killer
3,35704,2015,02-10-2015,shark lake
4,31992,2015,19-07-2015,sharktopus vs. whalewolf
5,30837,2015,22-07-2015,sharknado 3: oh hell no!
6,36355,2015,06-08-2015,piranha sharks
7,38988,2015,04-11-2015,sharkansas women's prison massacre
8,37053,2015,23-07-2015,roboshark
9,23838,2014,02-08-2014,sharktopus vs pteracuda


In [466]:
fd6 = fd4.groupby(["Años"]).count().dropna().reset_index()

fd6


,Años,Lanzamiento,Título
0,1932,1,1
1,1936,1,1
2,1950,1,1
3,1958,1,1
4,1969,1,1
5,1975,1,1
6,1976,2,2
7,1978,1,1
8,1981,2,2
9,1983,1,1


In [467]:
asd = df5[["Años", "Caso","Mujeres"]]

asd.head(2)

,Años,Caso,Mujeres
0,1848,1,0
1,1900,11,0


In [468]:
qwerty = fd6[["Años","Lanzamiento","Título"]]

qwerty.head(2)

,Años,Lanzamiento,Título
0,1932,1,1
1,1936,1,1


In [473]:
dtest = pd.merge(asd, qwerty, on="Años")

dtest

,Años,Caso,Mujeres,Lanzamiento,Título
0,1932,25,2,1,1
1,1936,23,1,1,1
2,1950,39,0,1,1
3,1958,47,4,1,1
4,1969,24,0,1,1
5,1975,44,5,1,1
6,1976,27,2,2,2
7,1978,22,2,1,1
8,1981,42,3,2,2
9,1983,46,5,1,1


In [474]:
dfinal = pd.merge(df5, fd6, on="Años")

dfinal

,Años,Caso,Hombres,Mujeres,Mortalidad,Lanzamiento,Título
0,1932,25,23,2,8,1,1
1,1936,23,22,1,8,1,1
2,1950,39,39,0,12,1,1
3,1958,47,43,4,19,1,1
4,1969,24,24,0,0,1,1
5,1975,44,39,5,7,1,1
6,1976,27,25,2,6,2,2
7,1978,22,20,2,4,1,1
8,1981,42,39,3,8,2,2
9,1983,46,41,5,7,1,1
